.mp3 to .wav

In [1]:
%pip install pydub

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pydub import AudioSegment

In [3]:
DATA_PATH = "./"
audio_file_name = "NceS - Burn"

In [4]:
# mp3 to wav
audio = AudioSegment.from_mp3(DATA_PATH + audio_file_name + ".mp3")
audio = audio.set_channels(1)
audio.export(DATA_PATH + audio_file_name + ".wav", format = "wav")

<_io.BufferedRandom name='./NceS - Burn.wav'>

Qwen-Audio

In [5]:
%pip install -r Qwen_requirements.txt

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
import os
torch.manual_seed(42)


In [7]:
MODEL_PATH = "/ssdshare/LLMs/"
PWD = os.getcwd()
DATA_PATH = './'
if DATA_PATH[0] == '.' :
    DATA_PATH = PWD + "/" + DATA_PATH
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "WHY DONT YOU HAVE CUDA???????"
TEMPORARY_PATH = DATA_PATH + ".tmp/"
if not os.path.exists(TEMPORARY_PATH) :
    os.makedirs(TEMPORARY_PATH)
CUDA_NUM = torch.cuda.device_count()
if CUDA_NUM == 1 :
    print("WARNING : Only 1 GPU may cause some problems.")
CUDA_DEVICE = [f"cuda:{i}" for i in range(CUDA_NUM)]
WINDOW_SIZE = 30_000
OVERLAP_SIZE = 5_000
OUTPUT_FILE = "extract_output.txt"


In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH + "Qwen-Audio-Chat/", trust_remote_code = True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="auto", trust_remote_code=True, fp16=True).eval()
models = []
for device in CUDA_DEVICE :
    model = AutoModelForCausalLM.from_pretrained(MODEL_PATH + "Qwen-Audio-Chat/", device_map = device, trust_remote_code = True).eval()
    models.append(model)



audio_start_id: 155163, audio_end_id: 155164, audio_pad_id: 151851.


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [9]:
query = tokenizer.from_list_format([
    {'audio': DATA_PATH + audio_file_name + '.wav'}, # Either a local path or an url
    {'text': 'Please give a detailed description of this piece of music, with no less than 5 sentences. Remind that you should give the description of the music, not the lyrics.'},
])
response, history = models[0].chat(tokenizer, query=query, history=None)
print(response)
# The person says: "mister quilter is the apostle of the middle classes and we are glad to welcome his gospel".

# 2nd dialogue turn
response, history = models[0].chat(tokenizer, 'Extract all the lyrics if the music has.', history=history)
print(response)
# The word "middle classes" starts at <|2.33|> seconds and ends at <|3.26|> seconds.

A pop/EDM instrumental with synthesizers, acoustic guitars, and a catchy beat. The lyrics are about living life to the fullest and chasing dreams.
The lyrics are:"Living in a crowded place, searching for the quiet that you're not to breathe, Give up upon your sanctuary, hide behind your shadow, While you try to take the sun down, ought to never change your gold, After thinking that you're in the world of no one, No one ever told you that you have to fight for something, Or your life will be too cold".


Partition&Describe

In [10]:
def meaningful_lyrics(lyrics) :
    if "NOLYRICS" in lyrics :
        return False
    if len(lyrics) < 100 :
        return False
    return True

def extract(file_name, device = 0, path = DATA_PATH) :
    query = tokenizer.from_list_format([
        {'audio': path + file_name + '.wav'}, 
        {'text': 'Please give a detailed description (emotion, background) of this piece of music, with no less than 5 sentences. You should give 5 sentences, not the lyrics, not words. '},
    ])
    decription, _ = models[device].chat(tokenizer, query = query, history = None)

    query = tokenizer.from_list_format([
        {'audio': path + file_name + '.wav'},
        {'text': 'Extract all the lyrics of this music if it has. Say "NOLYRICS" if it does not have lyrics or the lyrics are meaningless.'},
    ])
    
    lyrics, _ = models[device].chat(tokenizer, query = query, history = None)
    if not meaningful_lyrics(lyrics) :
        lyrics = None
    else :
        lyrics = lyrics.split('"')[1]
    return decription, lyrics

# test
# description, lyrics = extract("HyuN - Infinity Heaven")
# print(description)
# print(lyrics)
# description, lyrics = extract("NceS - Burn")
# print(description)
# print(lyrics)

In [11]:
from pydub import AudioSegment
from scipy.io.wavfile import read, write


In [12]:
def partition_extract(file_name, no_clear = False) :
    #####################################
    # I don't recommend parallel because running the same model continiously causes problems.
    #####################################
    file_name = file_name + ''
    audio = AudioSegment.from_wav(DATA_PATH + file_name + ".wav")
    duration = len(audio)
    num_of_pieces = (duration - OVERLAP_SIZE) // (WINDOW_SIZE - OVERLAP_SIZE) + 1
    file_name = file_name.replace(" ", "_")
    if not os.path.exists(TEMPORARY_PATH) :
        os.makedirs(TEMPORARY_PATH)
    if not os.path.exists(TEMPORARY_PATH + file_name) :
        os.makedirs(TEMPORARY_PATH + file_name)

    description = []
    lyrics = []


    for i in range(num_of_pieces) :
        start = i * (WINDOW_SIZE - OVERLAP_SIZE)
        end = start + WINDOW_SIZE
        if end > duration :
            end = duration
        piece = audio[start:end]
        piece.export(TEMPORARY_PATH + file_name + f"/{i}.wav", format = "wav")
        description_piece, lyrics_piece = extract(f"/{i}", device = i % CUDA_NUM, path = TEMPORARY_PATH + file_name + "/")
        description.append(description_piece)
        lyrics.append(lyrics_piece)

    if not no_clear :
        os.system(f"rm -rf {TEMPORARY_PATH + file_name + '/'}")

    return description, lyrics

# test
# description, lyrics = partition_extract('NceS - Burn')
# print(description)
# print(lyrics)
# description, lyrics = partition_extract('HyuN - Infinity Heaven')
# print(description)
# print(lyrics)


In [13]:
description, lyrics = partition_extract(audio_file_name)
prompt = f"This music is cut into {len(description)} pieces. Each piece has a length of {WINDOW_SIZE // 1000} seconds and an overlap of {OVERLAP_SIZE // 1000} seconds. The description of each piece is as follows:\n"
for i, d in enumerate(description) :
    prompt += f"Description piece {i + 1}: {d}\n"

have_lyrics = False
for l in lyrics :
    if l is not None :
        have_lyrics = True
        break

if have_lyrics :
    prompt += f"\nThe lyrics are as follows:\n"
    for i, l in enumerate(lyrics) :
        if l is not None :
            prompt += f"{l}\n"


In [14]:
with open(TEMPORARY_PATH + OUTPUT_FILE, "w") as f :
    f.write(prompt)